In [55]:
import pandas as pd
import numpy as np
from datetime import datetime 
#importing data from 
from google.colab import drive
from google.colab import files
drive.mount("/content/drive")
%cd ../
%cd /content/drive/Shared\ drives/Maleen_Kidiwela_Shared_Drive/AI/train
!pwd
!ls

In [2]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

In [3]:
conditions = pd.read_csv("conditions.csv")
patients = pd.read_csv("patients.csv")
observations = pd.read_csv("observations.csv")
careplans = pd.read_csv("careplans.csv")
encounters = pd.read_csv("encounters.csv")
devices = pd.read_csv("devices.csv")
supplies = pd.read_csv("supplies.csv")
procedures = pd.read_csv("procedures.csv")
medications = pd.read_csv("medications.csv")

In [ ]:

ID = list(patients.Id)
IDenc = list(encounters.PATIENT)
birth = list(patients.BIRTHDATE)
death = list(patients.DEATHDATE)
race = list(patients.RACE)
ethnicity = list(patients.ETHNICITY)
enc_start = list(encounters.START)
enc_end = list(encounters.STOP)
enc_code = list(encounters.CODE)
coverage = list(patients.HEALTHCARE_COVERAGE)

covid_patient_id=conditions[conditions.CODE ==840539006].PATIENT.unique()
negative_covid_patient_ids = observations[(observations.CODE =='94531-1')& (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
inpatients_ids = encounters[(encounters.REASONCODE == 840539006) & (encounters.CODE == 1505002)].PATIENT
deceased_ids = np.intersect1d(covid_patient_id,patients[patients.DEATHDATE.notna()].Id)
vent_ids = procedures[(procedures.CODE == 26763009)&(procedures.PATIENT.isin(covid_patient_id))].PATIENT
icu_ids = encounters[(encounters.CODE == 305351004)&(encounters.PATIENT.isin(covid_patient_id))].PATIENT

HIV_ids = observations[(observations.DESCRIPTION=='HIV status') & (observations.VALUE=='HIV positive') & (observations.PATIENT.isin(ID))].PATIENT.unique()
stroke_ids =conditions[(conditions.DESCRIPTION=='stroke') & (conditions.PATIENT.isin(ID))].PATIENT
cardiac_ids = conditions[conditions.DESCRIPTION.isin(['History of cardiac arrest (situation)', 'Cardiac Arrest']) & (conditions.PATIENT.isin(ID))].PATIENT
lungD1=['Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)','Small cell carcinoma of lung (disorder)','Suspected lung cancer (situation)']
lungD2=['Non-small cell carcinoma of lung  TNM stage 1 (disorder)','Non-small cell carcinoma of lung  TNM stage 2 (disorder)','Non-small cell lung cancer (disorder)']
lungD1_ids = conditions[(conditions.DESCRIPTION.isin(lungD1))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
lungD2_ids =conditions[(conditions.DESCRIPTION.isin(lungD2))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
asthma_ids =conditions[(conditions.DESCRIPTION=='Childhood asthma')& (conditions.STOP!=conditions.STOP) & (conditions.PATIENT.isin(ID))].PATIENT.unique()
heartC_ids=conditions[(conditions.DESCRIPTION.isin(['Injury of heart (disorder)', 'Chronic congestive heart failure (disorder)']))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
obeselvl1_ids = conditions[(conditions.DESCRIPTION=='Body mass index 30+ - obesity (finding)') & (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
obseslvl2_ids = conditions[(conditions.DESCRIPTION=='Body mass index 40+ - severely obese (finding)')& (conditions.STOP!=conditions.STOP) & (conditions.PATIENT.isin(ID))].PATIENT.unique()
diabetes_cond=['Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)','Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)',
               'Proteinuria due to type 2 diabetes mellitus (disorder)','Blindness due to type 2 diabetes mellitus (disorder)','Microalbuminuria due to type 2 diabetes mellitus (disorder)',
               'Neuropathy due to type 2 diabetes mellitus (disorder)','Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)',
               'Diabetic retinopathy associated with type II diabetes mellitus (disorder)', 'Prediabetes',]

diabetes_ids=conditions[(conditions.DESCRIPTION.isin(diabetes_cond))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
kidneystage1_ids=conditions[(conditions.DESCRIPTION.isin(['Chronic kidney disease stage 2 (disorder)', 'Injury of kidney (disorder)'])) & (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
kidneystage2_ids=conditions[(conditions.DESCRIPTION=='Chronic kidney disease stage 2 (disorder)')& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
kidneystage3_ids=conditions[(conditions.DESCRIPTION=='Chronic kidney disease stage 3 (disorder)')& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()

encounters.DESCRIPTION[13]
x_train = pd.DataFrame()
y_train = pd.DataFrame()

a = list(range(0,10000)) #len(ID)

#here we perform feature selection
for x in a:
  #x_train.loc[x,'ID'] = ID[x]
  #y_train.loc[x,'ID'] = ID[x]

  if race[x] == 'white':
    x_train.loc[x,'race'] = 1
  elif race[x] == 'asian':
    x_train.loc[x,'race'] = 2
  elif race[x] == 'black':
    x_train.loc[x,'race'] = 3

  if ethnicity == 'hispanic':
    x_train.loc[x,'ethnicity'] = 1
  else:
    x_train.loc[x,'ethnicity'] = 0

  x_train.loc[x,'coverage'] = coverage[x]

# 
  if ID[x] in HIV_ids:
    x_train.loc[x,'HIV_Status'] = 1
  else:
    x_train.loc[x,'HIV_Status'] = 0

  if ID[x] in stroke_ids:
    x_train.loc[x,'stroke_Status'] = 1
  else:
    x_train.loc[x,'stroke_Status'] = 0

  if ID[x] in cardiac_ids:
    x_train.loc[x,'cardiac_Status'] = 1
  else:
    x_train.loc[x,'cardiac_Status'] = 0

  if ID[x] in lungD1_ids:
    x_train.loc[x,'lung_Status'] = 1
  elif ID[x] in lungD2_ids:
    x_train.loc[x,'lung_Status'] = 2
  else:
    x_train.loc[x,'lung_Status'] = 0

  if ID[x] in heartC_ids:
    x_train.loc[x,'heart_Status'] = 1
  else:
    x_train.loc[x,'heart_Status'] = 0

  if ID[x] in asthma_ids:
    x_train.loc[x,'asthma_Status'] = 1
  else:
    x_train.loc[x,'asthma_Status'] = 0

  if ID[x] in diabetes_ids:
    x_train.loc[x,'diabetes_Status'] = 1
  else:
    x_train.loc[x,'diabetes_Status'] = 0

  if ID[x] in obeselvl1_ids:
    x_train.loc[x,'obesity_Status'] = 1
  elif ID[x] in obseslvl2_ids:
    x_train.loc[x,'obesity_Status'] = 2
  else:
    x_train.loc[x,'obesity_Status'] = 0

  if ID[x] in kidneystage1_ids:
    x_train.loc[x,'kidney_Status'] = 1
  elif ID[x] in kidneystage2_ids:
    x_train.loc[x,'kidney_Status'] = 2
  elif ID[x] in kidneystage3_ids:
    x_train.loc[x,'kidney_Status'] = 3
  else:
    x_train.loc[x,'kidney_Status'] = 0

  if ID[x] in covid_patient_id:
    y_train.loc[x,'Covid_Status'] = 1
  elif ID[x] in negative_covid_patient_ids:
    y_train.loc[x,'Covid_Status'] = 0
  else:
    y_train.loc[x,'Covid_Status'] = 0

  if ID[x] not in deceased_ids:
    y_train.loc[x,'Alive_Status'] = 1
    b_date = datetime.strptime(birth[x], '%Y-%m-%d')
    d_date = datetime.today()
    x_train.loc[x,'age']= int((d_date-b_date).days/365)
  else:
    y_train.loc[x,'Alive_Status'] = 0
    b_date = datetime.strptime(birth[x], '%Y-%m-%d')
    d_date = datetime.strptime(death[x], '%Y-%m-%d')
    x_train.loc[x,'age']= int((d_date-b_date).days/365)

  if ID[x] in inpatients_ids:
    y_train.loc[x,'COVID_Hospitalization'] = 1
    ind0 = get_index_positions(IDenc,ID[x])
    ind1 = get_index_positions(enc_code,1505002)
    ind2 = list(set(ind0) & set(ind2))
    if not not ind2:
      b = datetime.strptime((enc_end[int(ind2[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      c = datetime.strptime((enc_start[int(ind2[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      y_train.loc[x,'DaysinHospitalization'] = round(((b-c).seconds//3600)/24,2)
    else:
      y_train.loc[x,'DaysinHospitalization'] = 0
  else:
    y_train.loc[x,'COVID_Hospitalization'] = 0
    y_train.loc[x,'DaysinHospitalization'] = 0

  if ID[x] in vent_ids:
    y_train.loc[x,'Ventilator_Status'] = 1
  else:
    y_train.loc[x,'Ventilator_Status'] = 0

  if ID[x] in icu_ids:
    y_train.loc[x,'ICU_Status'] = 1
    ind3 = get_index_positions(IDenc,ID[x])
    ind4 = get_index_positions(enc_code,305351004)
    ind5 = list(set(ind3) & set(ind4))
    if not not ind5:
      d = datetime.strptime((enc_end[int(ind5[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      e = datetime.strptime((enc_start[int(ind5[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      y_train.loc[x,'DaysinICU'] = round(((d-e).seconds//3600)/24,3)
    else:
      y_train.loc[x,'DaysinICU'] = 0
  else:
    y_train.loc[x,'ICU_Status'] = 0
    y_train.loc[x,'DaysinICU'] = 0



In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd ../
%cd /content/drive/Shared\ drives/Maleen_Kidiwela_Shared_Drive/AI/test
!pwd
!ls

conditions = pd.read_csv("conditions.csv")
patients = pd.read_csv("patients.csv")
observations = pd.read_csv("observations.csv")
careplans = pd.read_csv("careplans.csv")
encounters = pd.read_csv("encounters.csv")
devices = pd.read_csv("devices.csv")
supplies = pd.read_csv("supplies.csv")
procedures = pd.read_csv("procedures.csv")
medications = pd.read_csv("medications.csv")

ID = list(patients.Id)
IDenc = list(encounters.PATIENT)
birth = list(patients.BIRTHDATE)
death = list(patients.DEATHDATE)
race = list(patients.RACE)
ethnicity = list(patients.ETHNICITY)
enc_start = list(encounters.START)
enc_end = list(encounters.STOP)
enc_code = list(encounters.CODE)
coverage = list(patients.HEALTHCARE_COVERAGE)

covid_patient_id=conditions[conditions.CODE ==840539006].PATIENT.unique()
negative_covid_patient_ids = observations[(observations.CODE =='94531-1')& (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
inpatients_ids = encounters[(encounters.REASONCODE == 840539006) & (encounters.CODE == 1505002)].PATIENT
deceased_ids = np.intersect1d(covid_patient_id,patients[patients.DEATHDATE.notna()].Id)
vent_ids = procedures[(procedures.CODE == 26763009)&(procedures.PATIENT.isin(covid_patient_id))].PATIENT
icu_ids = encounters[(encounters.CODE == 305351004)&(encounters.PATIENT.isin(covid_patient_id))].PATIENT

HIV_ids = observations[(observations.DESCRIPTION=='HIV status') & (observations.VALUE=='HIV positive') & (observations.PATIENT.isin(ID))].PATIENT.unique()
stroke_ids =conditions[(conditions.DESCRIPTION=='stroke') & (conditions.PATIENT.isin(ID))].PATIENT
cardiac_ids = conditions[conditions.DESCRIPTION.isin(['History of cardiac arrest (situation)', 'Cardiac Arrest']) & (conditions.PATIENT.isin(ID))].PATIENT
lungD1=['Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)','Small cell carcinoma of lung (disorder)','Suspected lung cancer (situation)']
lungD2=['Non-small cell carcinoma of lung  TNM stage 1 (disorder)','Non-small cell carcinoma of lung  TNM stage 2 (disorder)','Non-small cell lung cancer (disorder)']
lungD1_ids = conditions[(conditions.DESCRIPTION.isin(lungD1))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
lungD2_ids =conditions[(conditions.DESCRIPTION.isin(lungD2))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
asthma_ids =conditions[(conditions.DESCRIPTION=='Childhood asthma')& (conditions.STOP!=conditions.STOP) & (conditions.PATIENT.isin(ID))].PATIENT.unique()
heartC_ids=conditions[(conditions.DESCRIPTION.isin(['Injury of heart (disorder)', 'Chronic congestive heart failure (disorder)']))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
obeselvl1_ids = conditions[(conditions.DESCRIPTION=='Body mass index 30+ - obesity (finding)') & (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
obseslvl2_ids = conditions[(conditions.DESCRIPTION=='Body mass index 40+ - severely obese (finding)')& (conditions.STOP!=conditions.STOP) & (conditions.PATIENT.isin(ID))].PATIENT.unique()
diabetes_cond=['Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)','Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)',
               'Proteinuria due to type 2 diabetes mellitus (disorder)','Blindness due to type 2 diabetes mellitus (disorder)','Microalbuminuria due to type 2 diabetes mellitus (disorder)',
               'Neuropathy due to type 2 diabetes mellitus (disorder)','Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)',
               'Diabetic retinopathy associated with type II diabetes mellitus (disorder)', 'Prediabetes',]

diabetes_ids=conditions[(conditions.DESCRIPTION.isin(diabetes_cond))& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
kidneystage1_ids=conditions[(conditions.DESCRIPTION.isin(['Chronic kidney disease stage 2 (disorder)', 'Injury of kidney (disorder)'])) & (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
kidneystage2_ids=conditions[(conditions.DESCRIPTION=='Chronic kidney disease stage 2 (disorder)')& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()
kidneystage3_ids=conditions[(conditions.DESCRIPTION=='Chronic kidney disease stage 3 (disorder)')& (conditions.STOP!=conditions.STOP)& (conditions.PATIENT.isin(ID))].PATIENT.unique()

encounters.DESCRIPTION[13]
x_test = pd.DataFrame()
y_test = pd.DataFrame()

a = list(range(0,len(ID))) #len(ID)

#here we perform feature selection
for x in a:
  #x_train.loc[x,'ID'] = ID[x]
  #y_train.loc[x,'ID'] = ID[x]

  if race[x] == 'white':
    x_test.loc[x,'race'] = 1
  elif race[x] == 'asian':
    x_test.loc[x,'race'] = 2
  elif race[x] == 'black':
    x_test.loc[x,'race'] = 3

  if ethnicity == 'hispanic':
    x_test.loc[x,'ethnicity'] = 1
  else:
    x_test.loc[x,'ethnicity'] = 0

  x_test.loc[x,'coverage'] = coverage[x]

# 
  if ID[x] in HIV_ids:
    x_test.loc[x,'HIV_Status'] = 1
  else:
    x_test.loc[x,'HIV_Status'] = 0

  if ID[x] in stroke_ids:
    x_test.loc[x,'stroke_Status'] = 1
  else:
    x_test.loc[x,'stroke_Status'] = 0

  if ID[x] in cardiac_ids:
    x_test.loc[x,'cardiac_Status'] = 1
  else:
    x_test.loc[x,'cardiac_Status'] = 0

  if ID[x] in lungD1_ids:
    x_test.loc[x,'lung_Status'] = 1
  elif ID[x] in lungD2_ids:
    x_test.loc[x,'lung_Status'] = 2
  else:
    x_test.loc[x,'lung_Status'] = 0

  if ID[x] in heartC_ids:
    x_test.loc[x,'heart_Status'] = 1
  else:
    x_test.loc[x,'heart_Status'] = 0

  if ID[x] in asthma_ids:
    x_test.loc[x,'asthma_Status'] = 1
  else:
    x_test.loc[x,'asthma_Status'] = 0

  if ID[x] in diabetes_ids:
    x_test.loc[x,'diabetes_Status'] = 1
  else:
    x_test.loc[x,'diabetes_Status'] = 0

  if ID[x] in obeselvl1_ids:
    x_test.loc[x,'obesity_Status'] = 1
  elif ID[x] in obseslvl2_ids:
    x_test.loc[x,'obesity_Status'] = 2
  else:
    x_test.loc[x,'obesity_Status'] = 0

  if ID[x] in kidneystage1_ids:
    x_test.loc[x,'kidney_Status'] = 1
  elif ID[x] in kidneystage2_ids:
    x_test.loc[x,'kidney_Status'] = 2
  elif ID[x] in kidneystage3_ids:
    x_test.loc[x,'kidney_Status'] = 3
  else:
    x_test.loc[x,'kidney_Status'] = 0

  if ID[x] in covid_patient_id:
    y_test.loc[x,'Covid_Status'] = 1
  elif ID[x] in negative_covid_patient_ids:
    y_test.loc[x,'Covid_Status'] = 0
  else:
    y_test.loc[x,'Covid_Status'] = 0

  if ID[x] not in deceased_ids:
    y_test.loc[x,'Alive_Status'] = 1
    b_date = datetime.strptime(birth[x], '%Y-%m-%d')
    d_date = datetime.today()
    x_test.loc[x,'age']= int((d_date-b_date).days/365)
  else:
    y_test.loc[x,'Alive_Status'] = 0
    b_date = datetime.strptime(birth[x], '%Y-%m-%d')
    d_date = datetime.strptime(death[x], '%Y-%m-%d')
    x_test.loc[x,'age']= int((d_date-b_date).days/365)

  if ID[x] in inpatients_ids:
    y_test.loc[x,'COVID_Hospitalization'] = 1
    ind0 = get_index_positions(IDenc,ID[x])
    ind1 = get_index_positions(enc_code,1505002)
    ind2 = list(set(ind0) & set(ind2))
    if not not ind2:
      b = datetime.strptime((enc_end[int(ind2[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      c = datetime.strptime((enc_start[int(ind2[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      y_test.loc[x,'DaysinHospitalization'] = round(((b-c).seconds//3600)/24,2)
    else:
      y_test.loc[x,'DaysinHospitalization'] = 0
  else:
    y_test.loc[x,'COVID_Hospitalization'] = 0
    y_test.loc[x,'DaysinHospitalization'] = 0

  if ID[x] in vent_ids:
    y_test.loc[x,'Ventilator_Status'] = 1
  else:
    y_test.loc[x,'Ventilator_Status'] = 0

  if ID[x] in icu_ids:
    y_test.loc[x,'ICU_Status'] = 1
    ind3 = get_index_positions(IDenc,ID[x])
    ind4 = get_index_positions(enc_code,305351004)
    ind5 = list(set(ind3) & set(ind4))
    if not not ind5:
      d = datetime.strptime((enc_end[int(ind5[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      e = datetime.strptime((enc_start[int(ind5[0])]).replace("T"," ").replace('Z',''), '%Y-%m-%d %H:%M:%S')
      y_test.loc[x,'DaysinICU'] = round(((d-e).seconds//3600)/24,3)
    else:
      y_test.loc[x,'DaysinICU'] = 0
  else:
    y_test.loc[x,'ICU_Status'] = 0
    y_test.loc[x,'DaysinICU'] = 0



In [ ]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 16, activation = 'relu', input_dim = 13))
classifier.add(Dense(units = 8, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 7, activation = 'sigmoid'))

classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')
classifier.fit(x_train, y_train, batch_size = 1, epochs = 100)
y_pred = classifier.predict(x_test)
Y_Pred = pd.DataFrame(data=ID)

for i in list([range(2,7)]):
  y_pred.iloc[:,i] = [ 1 if y>=0.5 else 0 for y in y_pred.iloc[:,i] ]
  Y_Pred.iloc[:,1] = y_pred.iloc[:,i]
  Y_Pred.to_csv('ypred.csv')
  files.download('ypred.csv')
